In [301]:
import numpy as np

In [302]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [303]:
class NeuralNetwork:
    def __init__(self, num_inputs, num_hidden, num_outputs):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs

        # Initialize weights with small random numbers
        self.weights_input_hidden = np.random.randn(self.num_inputs, self.num_hidden)
        self.weights_hidden_output = np.random.randn(self.num_hidden, self.num_outputs)

    def feedforward(self, inputs):
        self.hidden = sigmoid(np.dot(inputs, self.weights_input_hidden))
        self.output = sigmoid(np.dot(self.hidden, self.weights_hidden_output))
        return self.output

    def backpropagate(self, inputs, actual_output, predicted_output, learning_rate):
        output_error = actual_output - predicted_output
        output_delta = output_error * sigmoid_derivative(predicted_output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden)

        self.weights_hidden_output += self.hidden.T.dot(output_delta) * learning_rate
        self.weights_input_hidden += inputs.T.dot(hidden_delta) * learning_rate

    def train(self, inputs, actual_output, epochs, learning_rate=0.1, patience=10):
        no_change_epochs = 0
        prev_weights_hidden_output = np.copy(self.weights_hidden_output)
        prev_weights_input_hidden = np.copy(self.weights_input_hidden)

        for epoch in range(epochs):
            predicted_output = self.feedforward(inputs)
            self.backpropagate(inputs, actual_output, predicted_output, learning_rate)

            # Check for weight changes
            if np.array_equal(prev_weights_hidden_output, self.weights_hidden_output) and np.array_equal(prev_weights_input_hidden, self.weights_input_hidden):
                no_change_epochs += 1
            else:
                no_change_epochs = 0
            
            prev_weights_hidden_output = np.copy(self.weights_hidden_output)
            prev_weights_input_hidden = np.copy(self.weights_input_hidden)

            if no_change_epochs >= patience:
                print(f"Training stopped early at epoch {epoch} due to no change in weights for {patience} consecutive epochs.")
                break

    def predict(self, inputs):
        output = self.feedforward(inputs)
        return sigmoid(output)

In [304]:
def create_dataset(num_inputs, gate):
    if gate == 'AND':
        inputs = np.array([[int(x) for x in format(i, f'0{num_inputs}b')] for i in range(2**num_inputs)])
        labels = np.array([[int(np.all(input_vec))] for input_vec in inputs])
    elif gate == 'OR':
        inputs = np.array([[int(x) for x in format(i, f'0{num_inputs}b')] for i in range(2**num_inputs)])
        labels = np.array([[int(np.any(input_vec))] for input_vec in inputs])
    elif gate == 'XOR':
        inputs = np.array([[int(x) for x in format(i, f'0{num_inputs}b')] for i in range(2**num_inputs)])
        labels = np.array([[int(np.sum(input_vec) % 2)] for input_vec in inputs])
    else:
        raise ValueError("Invalid gate type. Choose 'AND', 'OR', or 'XOR'.")
    return inputs, labels

In [305]:
num_inputs = 4
gate = 'XOR' 

In [306]:
inputs, labels = create_dataset(num_inputs, gate)
nn = NeuralNetwork(num_inputs=num_inputs, num_hidden=4, num_outputs=1)
epochs = 20000
nn.train(inputs, labels, epochs, learning_rate=0.1)

In [307]:
for i in range(len(inputs)):
    predicted_output = nn.predict(inputs[i])
    print(f"Input: {inputs[i]}, Predicted Output: {predicted_output}, Actual Output: {labels[i]}")

Input: [0 0 0 0], Predicted Output: [0.50848844], Actual Output: [0]
Input: [0 0 0 1], Predicted Output: [0.72879718], Actual Output: [1]
Input: [0 0 1 0], Predicted Output: [0.72832815], Actual Output: [1]
Input: [0 0 1 1], Predicted Output: [0.73073945], Actual Output: [1]
Input: [0 1 0 0], Predicted Output: [0.72828473], Actual Output: [1]
Input: [0 1 0 1], Predicted Output: [0.73073922], Actual Output: [1]
Input: [0 1 1 0], Predicted Output: [0.73062935], Actual Output: [1]
Input: [0 1 1 1], Predicted Output: [0.73081155], Actual Output: [1]
Input: [1 0 0 0], Predicted Output: [0.72824401], Actual Output: [1]
Input: [1 0 0 1], Predicted Output: [0.73073301], Actual Output: [1]
Input: [1 0 1 0], Predicted Output: [0.73062173], Actual Output: [1]
Input: [1 0 1 1], Predicted Output: [0.73081069], Actual Output: [1]
Input: [1 1 0 0], Predicted Output: [0.7305451], Actual Output: [1]
Input: [1 1 0 1], Predicted Output: [0.73080573], Actual Output: [1]
Input: [1 1 1 0], Predicted Output: